In [49]:
import yfinance as yf
import pandas as pd
import numpy as np
import pandas_ta as ta
import matplotlib.pyplot as plt

Extracting necessary data.
We will extract the 30 companies in the DJIA and extract the historical price data

In [50]:
url = "https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average"
tables = pd.read_html(url)
companies_table = tables[2] #where the 3rd table on the wiki page contains the 30 companies
companies_table.to_csv('companies_table.csv',index = False) #converting table into a csv file

#have a dataframe for the data for all 30 companies
data = pd.DataFrame()
for symbol in companies_table['Symbol']:
    company_data = yf.download(symbol, start='2014-01-01')
    #new column in the data where the symbol is included as well
    company_data["Symbol"] = symbol
    data = pd.concat([data,company_data])

data = data.reset_index()
data


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,2014-01-02,115.426422,116.220734,115.058525,115.493309,80.922089,3650312,MMM
1,2014-01-03,115.693977,116.120399,115.409698,115.760872,81.109566,2551427,MMM
2,2014-01-06,116.011703,116.463211,114.949837,115.075249,80.629166,3243672,MMM
3,2014-01-07,114.924751,115.551842,114.657188,115.091972,80.640892,3020498,MMM
4,2014-01-08,114.949837,115.175583,113.996658,114.239128,80.043327,3734749,MMM
...,...,...,...,...,...,...,...,...
80494,2024-10-25,83.199997,83.300003,82.440002,82.510002,82.510002,8959400,WMT
80495,2024-10-28,82.750000,83.080002,82.470001,82.750000,82.750000,7990200,WMT
80496,2024-10-29,82.669998,82.860001,81.669998,81.699997,81.699997,9682400,WMT
80497,2024-10-30,81.599998,81.830002,81.300003,81.389999,81.389999,9268800,WMT


Calculate daily returns (percentage change)

In [51]:
data['Daily Return'] = data.groupby('Symbol')['Adj Close'].pct_change()
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,Daily Return
0,2014-01-02,115.426422,116.220734,115.058525,115.493309,80.922089,3650312,MMM,NaN
1,2014-01-03,115.693977,116.120399,115.409698,115.760872,81.109566,2551427,MMM,0.002317
2,2014-01-06,116.011703,116.463211,114.949837,115.075249,80.629166,3243672,MMM,-0.005923
3,2014-01-07,114.924751,115.551842,114.657188,115.091972,80.640892,3020498,MMM,0.000145
4,2014-01-08,114.949837,115.175583,113.996658,114.239128,80.043327,3734749,MMM,-0.007410


Implement the strategy
We need to identify the top 10 biggest losers of the trading day

In [52]:
#we need to filter the df which skips the first day as there is not indication for daily return (filed will be null)
data = data.dropna()



In [53]:
#sort data by date and Daily Return
sorted_data = data.sort_values(by=['Date', 'Daily Return'])

#group by date and take the lowest 10 for each date
lowest_daily_returns = sorted_data.groupby('Date').head(10).reset_index(drop=True)

now that we can get the 10 lowest peforming for each day, we need to simulate the trades

The following code will be complex, so lets breakdown the logic to it
1. intialise the initial capital we will invest and a dataframe to store the results
2. get a list of unique dates
3. start a loop for all the unique dates
    a) select the 10 worst performers for the current day
    b) find amount to invest per stock for the current day
    c) create a loop for each stock to invest in
        a) get the buy price (today's adj close) and sell price (tomorrows adj close)
        b) get number of shares to buy (amt_per_stock/today_price)
        c) get what those number of shares are worth tomorrow
        d) update a counter for the total value of all these stocks

    d) update what the initial capital will be for the next day
    e) store the results

In [54]:
#The following code will be complex, so lets breakdown the logic to it
#1. intialise the initial capital we will invest and a dataframe to store the results
#2. get a list of unique dates
#3. start a loop for all the unique dates
 #   a) select the 10 worst performers for the current day
  #  b) find amount to invest per stock for the current day
   # c) create a loop for each stock to invest in
    #    a) get the buy price (today's adj close) and sell price (tomorrows adj close)
     #   b) get number of shares to buy (amt_per_stock/today_price)
      #  c) get what those number of shares are worth tomorrow
       # d) update a counter for the total value of all these stocks

   # d) update what the initial capital will be for the next day
    #e) store the results

In [55]:
copy_data = data.copy()
copy_lowest_daily_returns = lowest_daily_returns.copy()

In [56]:
initial_capital = 100000
results_no_fees = pd.DataFrame(columns=['Date', 'Capital'])

#sort data by date
lowest_daily_returns = lowest_daily_returns.sort_values('Date')
data = data.sort_values('Date')

#get unique dates
unique_dates = lowest_daily_returns['Date'].unique()

for i, current_date in enumerate(unique_dates[:-1]):  # Exclude last date
    next_date = unique_dates[i + 1]
    
    #select 10 worst performers for the current date
    current_day_losers = lowest_daily_returns[lowest_daily_returns['Date'] == current_date].nsmallest(10, 'Daily Return')
    
    # Calculate amount to invest per stock
    amount_per_stock = initial_capital / 10
    
    total_value = 0
    
    for _, stock in current_day_losers.iterrows():
        symbol = stock['Symbol']
        
        # Find buy and sell prices
        buy_price = stock['Adj Close']
        sell_price = data[(data['Date'] == next_date) & (data['Symbol'] == symbol)]['Adj Close'].values
        
        # Calculate number of shares and value
        shares = amount_per_stock / buy_price
        value = shares * sell_price[0]
        
        total_value += value
    
    # Update capital
    initial_capital = total_value if total_value > 0 else initial_capital
    
    new_data = {'Date': next_date, 'Capital': initial_capital}
    # Store results
    results_no_fees = pd.concat([results_no_fees,pd.DataFrame.from_records([new_data])], ignore_index=True)

In [57]:
results_no_fees

,Date,Capital
0,2014-01-06,99496.773058
1,2014-01-07,100575.237984
2,2014-01-08,100386.296466
3,2014-01-09,100328.578794
4,2014-01-10,100915.019859
...,...,...
2720,2024-10-25,422252.157405
2721,2024-10-28,427093.666767
2722,2024-10-29,426113.292186
2723,2024-10-30,424948.296744


Now how does this compare to, if we just bought and held in the DIA?
Before we compare this, we are going to simulate how trading fees impacts the returns we have,
as there is a 0.005 per share cost on IBKR for trading fees.

In [58]:
initial_capital = 100000
results_fees = pd.DataFrame(columns=['Date', 'Capital'])

total_fees = 0
#sort data by date
copy_lowest_daily_returns = copy_lowest_daily_returns.sort_values('Date')
copy_data = copy_data.sort_values('Date')

#get unique dates
unique_dates = copy_lowest_daily_returns['Date'].unique()

for i, current_date in enumerate(unique_dates[:-1]):  # Exclude last date
    next_date = unique_dates[i + 1]
    
    #select 10 worst performers for the current date
    current_day_losers = copy_lowest_daily_returns[copy_lowest_daily_returns['Date'] == current_date].nsmallest(10, 'Daily Return')
    
    # Calculate amount to invest per stock
    amount_per_stock = initial_capital / 10
    
    total_value = 0
    total_shares = 0
    
    for _, stock in current_day_losers.iterrows():
        symbol = stock['Symbol']
        
        # Find buy and sell prices
        buy_price = stock['Adj Close']
        sell_price = copy_data[(copy_data['Date'] == next_date) & (copy_data['Symbol'] == symbol)]['Adj Close'].values
        
        # Calculate number of shares and value
        shares = amount_per_stock / buy_price
        value = shares * sell_price[0]

        #calculate the fees for the particular stock
        fees_due = shares * 0.005
        #update the new value (capital) and subtract the fees from it
        total_value = total_value + value - fees_due

        #sum the total fees as we need to know how much total fees was paid in the end
        total_fees = fees_due + total_fees

    # Update capital
    initial_capital = total_value if total_value > 0 else initial_capital
    
    new_data = {'Date': next_date, 'Capital': initial_capital}
    # Store results
    results_fees = pd.concat([results_fees,pd.DataFrame.from_records([new_data])], ignore_index=True)

In [59]:
results_fees


,Date,Capital
0,2014-01-06,99480.296674
1,2014-01-07,100545.805136
2,2014-01-08,100344.046496
3,2014-01-09,100275.624992
4,2014-01-10,100844.284535
...,...,...
2720,2024-10-25,348713.293966
2721,2024-10-28,352696.504097
2722,2024-10-29,351873.527906
2723,2024-10-30,350896.997252


Now how does this compare to, if we just bought and held in the DIA? Or how does this compare to the SPY?

Lets get the performance metrics

In [60]:
trading_days = 252

#daily capital returns
results_no_fees['Capital Return'] = results_no_fees['Capital'].pct_change() 
results_fees['Capital Return'] = results_fees['Capital'].pct_change()

# Sharpe Ratio assuming risk-free rate = 0

def annual_return(return_list):
    returns = (1 + return_list.mean())**trading_days - 1
    return returns * 100.0

def annual_volatilty(return_list):
    vol = return_list.std() * (trading_days**0.5)
    return vol * 100.0

def sharpe_ratio(returns, vol):
    #assuming trade free return is 0
    return returns/vol 

annual_return_fees = annual_return(results_fees['Capital Return'])
annual_vol_fees = annual_volatilty(results_fees['Capital Return'])
sharpe_ratio_fees = sharpe_ratio(annual_return_fees,annual_vol_fees)

# Printing the calculated metrics
print(f"Annualized Return WITH FEES: {annual_return_fees:.2f}% ")
print(f"Annualized Volatility WITH FEES: {annual_vol_fees:.2f}% ")
print(f"Sharpe Ratio WITH FEES: {sharpe_ratio_fees:.2f} ")

Annualized Return WITH FEES: 14.51% 
Annualized Volatility WITH FEES: 19.57% 
Sharpe Ratio WITH FEES: 0.74 


In [61]:
dia_data = yf.download('DIA', start='2014-01-01')
dia_data['Daily Return'] = dia_data['Adj Close'].pct_change()

spy_data = yf.download('SPY',start = '2014-01-01')
spy_data['Daily Return'] = spy_data['Adj Close'].pct_change()

# Calculate the data for DIA
annual_return_dia = annual_return(dia_data['Daily Return'])
annual_volatility_dia = annual_volatilty(dia_data['Daily Return'])
sharpe_ratio_dia = sharpe_ratio(annual_return_dia,annual_volatility_dia)

#Calculate data points for SPY
annual_return_spy = annual_return(spy_data['Daily Return'])
annual_volatility_spy = annual_volatilty(spy_data['Daily Return'])
sharpe_ratio_spy = sharpe_ratio(annual_return_dia,annual_volatility_dia)

# Gathering calculated data for mean reversion strategy and DIA
info = {
    'Mean Reversion Strategy INCL FEES': [annual_return_fees, annual_vol_fees, sharpe_ratio_fees],
    'DIA': [annual_return_dia, annual_volatility_dia, sharpe_ratio_dia],
    'SPY': [annual_return_spy,annual_volatility_spy,sharpe_ratio_spy]
}

# Creating a pandas DataFrame with this data
df_info = pd.DataFrame(info, index=['Annual Return', 'Annual Std Dev', 'Sharpe Ratio'])

# Printing the DataFrame
print(df_info)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

                Mean Reversion Strategy INCL FEES        DIA        SPY
Annual Return                           14.506989  12.966771  14.710938
Annual Std Dev                          19.570760  16.991343  17.184420
Sharpe Ratio                             0.741258   0.763140   0.763140


In [62]:
import plotly.graph_objects as go

# Calculate Cumulative Returns by adding 1 to daily returns, then calculate the cumulative product
dia_data['Cumulative Return'] = (1 + dia_data['Daily Return']).cumprod()
# Calculate Portfolio Value by multiplying Cumulative Returns by initial capital
dia_data['Portfolio Value'] = dia_data['Cumulative Return'] * 100000

spy_data['Cumulative Return'] = (1+ spy_data['Daily Return']).cumprod()
spy_data['Portfolio Value'] = spy_data['Cumulative Return'] * 100000

# Create Line plots 
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=results_fees['Date'], 
    y=results_fees['Capital'], 
    mode='lines', 
    name='Mean Reversion Strategy'
))

fig.add_trace(go.Scatter(
    x=dia_data.index, 
    y=dia_data['Portfolio Value'], 
    mode='lines', 
    name='DIA ETF'
))

fig.add_trace(go.Scatter(
    x=spy_data.index,
    y=spy_data['Portfolio Value'],
    mode = 'lines',
    name = 'S&P 500'
))

# Add titles and labels
fig.update_layout(
    title='Growth of $100,000 Portfolio Over Time',
    xaxis_title='Date',
    yaxis_title='Portfolio Value ($)',
    legend_title='Strategy',
    autosize=False,
    width=1000, 
    height=500
)

In [71]:
results_fees['Return Factor'] = 1 + results_fees['Capital Return']

yearly_return = results_fees['Return Factor'].resample('Y').prod() - 1

yearly_return_df = pd.DataFrame({
    'Year': yearly_return.index.year,
    'Yearly Return': yearly_return.values
})

print(yearly_return_df)

    Year  Yearly Return
0   2014       0.147151
1   2015       0.121681
2   2016       0.104333
3   2017       0.235762
4   2018       0.162794
5   2019       0.268401
6   2020       0.190343
7   2021       0.173116
8   2022      -0.240672
9   2023       0.109946
10  2024       0.153428
